In [25]:
import pandas as pd
import numpy as np


train_data = pd.read_table('../data/oppo_round1_train_20180929.txt',
                           names=['prefix', 'query_prediction', 'title', 'tag', 'label'],
                           header=None, encoding='utf-8').astype(str)

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
train_31 = pd.read_csv('../data/train_31_3.csv')

In [2]:
error_loc = np.where(train_data['label'].values == '音乐')[0][0]

In [4]:
train_data.loc[1815101]

prefix                                                           花开花又
query_prediction             {"花开花又落": "0.635", "花开花又落是什么歌": "0.365"}
title               大雁听过我的歌%2C山丹丹花开花又落....一年又一年?谁听过这首歌%2C知道歌名吗?\t知...
tag                                                           等你花开花又落
label                                                              音乐
Name: 1815101, dtype: object

In [5]:
train_4_dict_df = pd.read_csv('../data/train_vec_4.csv')
train_4_dict_df_nofix = pd.read_csv('../data/train_vec_4_no_fix.csv')

In [7]:
train_4_dict_df.loc[1815100:1815103]

,ws_similarity,maximum_similarity,median_similarity,mean_similarity
1815100,0.045656,0.214867,0.149781,0.097531
1815101,0.398330,0.671187,0.205169,0.267424
1815102,0.101104,0.214714,0.008437,0.027830
1815103,0.945661,1.000000,0.407879,0.491249


In [8]:
train_4_dict_df_nofix.loc[1815100:1815103]

,ws_similarity,maximum_similarity,median_similarity,mean_similarity
1815100,0.045656,0.214867,0.149781,0.097531
1815101,0.104817,1.000000,0.455706,0.455706
1815102,0.398330,0.671187,0.205169,0.267424
1815103,0.101104,0.214714,0.008437,0.027830


In [36]:
train_data_fix = train_data[train_data['label'] != '音乐']

In [39]:
train_data_fix.loc[1815100:1815103]

,prefix,query_prediction,title,tag,label
1815100,习仲勋,"{""习仲勋陵园旅游"": ""0.005"", ""习仲勋故居"": ""0.006"", ""习仲勋陵园""...",习仲勋传,百科,0
1815101,田螺,"{""田螺姑娘的故事"": ""0.011"", ""田螺怎么洗最快最干净"": ""0.068"", ""田...",田螺,百科,0
1815102,经期,"{""经期可以吃什么水果"": ""0.006"", ""经期减肥"": ""0.016"", ""经期可以吃...",经期助手,应用,0
1815103,花样多,"{""花样多的四字词语"": ""0.082"", ""花样多的成语是什么"": ""0.100"", ""花...",形容花样多的成语,知道,1


In [38]:
train_data_fix = train_data_fix.reset_index(drop=True)

In [11]:
train_data_fix['ws_similarity'] = train_4_dict_df_nofix['ws_similarity']

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
train_data_fix.loc[1815100:1815103]

,prefix,query_prediction,title,tag,label,ws_similarity
1815100,习仲勋,"{""习仲勋陵园旅游"": ""0.005"", ""习仲勋故居"": ""0.006"", ""习仲勋陵园""...",习仲勋传,百科,0,0.045656
1815102,田螺,"{""田螺姑娘的故事"": ""0.011"", ""田螺怎么洗最快最干净"": ""0.068"", ""田...",田螺,百科,0,0.398330
1815103,经期,"{""经期可以吃什么水果"": ""0.006"", ""经期减肥"": ""0.016"", ""经期可以吃...",经期助手,应用,0,0.101104


In [13]:
train_data_fix = pd.concat([train_data_fix, train_4_dict_df], axis=1)

In [15]:
train_data_fix.loc[1815100:1815103]

,prefix,query_prediction,title,tag,label,ws_similarity,ws_similarity,maximum_similarity,median_similarity,mean_similarity
1815100,习仲勋,"{""习仲勋陵园旅游"": ""0.005"", ""习仲勋故居"": ""0.006"", ""习仲勋陵园""...",习仲勋传,百科,0,0.045656,0.045656,0.214867,0.149781,0.097531
1815101,NaN,NaN,NaN,NaN,NaN,NaN,0.398330,0.671187,0.205169,0.267424
1815102,田螺,"{""田螺姑娘的故事"": ""0.011"", ""田螺怎么洗最快最干净"": ""0.068"", ""田...",田螺,百科,0,0.398330,0.101104,0.214714,0.008437,0.027830
1815103,经期,"{""经期可以吃什么水果"": ""0.006"", ""经期减肥"": ""0.016"", ""经期可以吃...",经期助手,应用,0,0.101104,0.945661,1.000000,0.407879,0.491249


In [17]:
train_31.loc[1815100:1815103]

,new_tag0,new_tag1,new_tag2,new_tag3,new_tag4,new_tag5,new_tag6,new_tag7,new_tag8,new_tag9,...,new_tag24,new_tag25,new_tag26,new_tag27,new_tag28,new_tag29,new_tag30,new_tag31,new_tag32,new_tag33
1815100,0.350088,0.295902,0.257198,0.446515,0.266337,0.282766,0.377607,0.291182,0.459474,0.000000,...,0.048309,0.038647,0.028986,0.024155,0.024155,0.000000,0.336341,0.459474,0.000000,0.338111
1815101,0.185646,0.325771,0.613103,0.535725,0.614886,0.241364,0.850772,0.447556,0.797000,0.322508,...,0.065823,0.035443,0.035443,0.030380,0.027848,0.027848,0.493433,0.850772,0.185646,0.357911
1815102,0.234428,0.624232,0.268512,0.247083,0.285568,0.534042,0.248578,0.275756,0.286709,0.217961,...,0.076923,0.059829,0.059829,0.051282,0.051282,0.051282,0.322287,0.624232,0.217961,0.319082
1815103,0.916361,0.786683,0.809261,0.910182,0.823241,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.092031,0.000000,0.000000,0.000000,0.000000,0.000000,0.849146,0.916361,0.000000,0.875554


In [18]:
train_31.shape

(1999998, 34)

In [23]:
train_data_fix = pd.concat([train_data_fix, train_31], axis=1)

In [24]:
train_data_fix.loc[1815100:1815103]

,prefix,query_prediction,title,tag,label,new_tag0,new_tag1,new_tag2,new_tag3,new_tag4,...,new_tag24,new_tag25,new_tag26,new_tag27,new_tag28,new_tag29,new_tag30,new_tag31,new_tag32,new_tag33
1815100,习仲勋,"{""习仲勋陵园旅游"": ""0.005"", ""习仲勋故居"": ""0.006"", ""习仲勋陵园""...",习仲勋传,百科,0,0.350088,0.295902,0.257198,0.446515,0.266337,...,0.048309,0.038647,0.028986,0.024155,0.024155,0.000000,0.336341,0.459474,0.000000,0.338111
1815101,NaN,NaN,NaN,NaN,NaN,0.185646,0.325771,0.613103,0.535725,0.614886,...,0.065823,0.035443,0.035443,0.030380,0.027848,0.027848,0.493433,0.850772,0.185646,0.357911
1815102,田螺,"{""田螺姑娘的故事"": ""0.011"", ""田螺怎么洗最快最干净"": ""0.068"", ""田...",田螺,百科,0,0.234428,0.624232,0.268512,0.247083,0.285568,...,0.076923,0.059829,0.059829,0.051282,0.051282,0.051282,0.322287,0.624232,0.217961,0.319082
1815103,经期,"{""经期可以吃什么水果"": ""0.006"", ""经期减肥"": ""0.016"", ""经期可以吃...",经期助手,应用,0,0.916361,0.786683,0.809261,0.910182,0.823241,...,0.092031,0.000000,0.000000,0.000000,0.000000,0.000000,0.849146,0.916361,0.000000,0.875554


In [ ]:
train_data[train_data['label'] != '音乐']

In [45]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score


def lgb_f1_score_sk(y_hat, y_true):
    y_true = np.round(y_true)
    y_hat = np.round(y_hat)  # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True


'''
Initial Configuration
'''
pd.set_option('display.expand_frame_repr', False)

'''
Data reading.
'''
train_data = pd.read_table('../data/oppo_round1_train_20180929.txt',
                           names=['prefix', 'query_prediction', 'title', 'tag', 'label'],
                           header=None, encoding='utf-8').astype(str)
val_data = pd.read_table('../data/oppo_round1_vali_20180929.txt',
                         names=['prefix', 'query_prediction', 'title', 'tag', 'label'],
                         header=None, encoding='utf-8').astype(str)
test_data = pd.read_table('../data/oppo_round1_test_A_20180929.txt',
                          names=['prefix', 'query_prediction', 'title', 'tag'],
                          header=None, encoding='utf-8').astype(str)

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
'''
Data preprocessing (clearing)
'''

print(np.where(train_data['label'].values == '音乐'))
train_data_noreset = train_data[train_data['label'] != '音乐']
train_data_reset = train_data_noreset.reset_index(drop=True)
train_data_noreset['label'] = train_data_noreset['label'].apply(lambda x: int(x))
train_data_reset['label'] = train_data_reset['label'].apply(lambda x: int(x))

(array([1815101], dtype=int64),)


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
train_data_noreset.loc[1815100:1815104]

,prefix,query_prediction,title,tag,label
1815100,习仲勋,"{""习仲勋陵园旅游"": ""0.005"", ""习仲勋故居"": ""0.006"", ""习仲勋陵园""...",习仲勋传,百科,0
1815102,田螺,"{""田螺姑娘的故事"": ""0.011"", ""田螺怎么洗最快最干净"": ""0.068"", ""田...",田螺,百科,0
1815103,经期,"{""经期可以吃什么水果"": ""0.006"", ""经期减肥"": ""0.016"", ""经期可以吃...",经期助手,应用,0
1815104,花样多,"{""花样多的四字词语"": ""0.082"", ""花样多的成语是什么"": ""0.100"", ""花...",形容花样多的成语,知道,1


In [49]:
train_data_reset.loc[1815100:1815104]

,prefix,query_prediction,title,tag,label
1815100,习仲勋,"{""习仲勋陵园旅游"": ""0.005"", ""习仲勋故居"": ""0.006"", ""习仲勋陵园""...",习仲勋传,百科,0
1815101,田螺,"{""田螺姑娘的故事"": ""0.011"", ""田螺怎么洗最快最干净"": ""0.068"", ""田...",田螺,百科,0
1815102,经期,"{""经期可以吃什么水果"": ""0.006"", ""经期减肥"": ""0.016"", ""经期可以吃...",经期助手,应用,0
1815103,花样多,"{""花样多的四字词语"": ""0.082"", ""花样多的成语是什么"": ""0.100"", ""花...",形容花样多的成语,知道,1
1815104,本命年,"{""本命年结婚好吗"": ""0.021"", ""本命年是什么意思"": ""0.025"", ""本命年...",本命年可以结婚吗……,健康,0


In [51]:
# semantic feature - 31_3

new_train_data = pd.read_csv('../data/train_31_3.csv')
train_data = pd.concat([train_data, new_train_data], axis=1)
train_data_noreset = pd.concat([train_data_noreset, new_train_data], axis=1)

# semantic feature - 4

train_data_dict_df = pd.read_csv('../data/train_vec_4.csv')
train_data = pd.concat([train_data, train_data_dict_df], axis=1)
train_data_noreset = pd.concat([train_data_noreset, train_data_dict_df], axis=1)

In [52]:
train_data_noreset.loc[1815100:1815104]

,prefix,query_prediction,title,tag,label,new_tag0,new_tag1,new_tag2,new_tag3,new_tag4,...,new_tag28,new_tag29,new_tag30,new_tag31,new_tag32,new_tag33,ws_similarity,maximum_similarity,median_similarity,mean_similarity
1815100,习仲勋,"{""习仲勋陵园旅游"": ""0.005"", ""习仲勋故居"": ""0.006"", ""习仲勋陵园""...",习仲勋传,百科,0.0,0.350088,0.295902,0.257198,0.446515,0.266337,...,0.024155,0.000000,0.336341,0.459474,0.000000,0.338111,0.045656,0.214867,0.149781,0.097531
1815101,NaN,NaN,NaN,NaN,NaN,0.185646,0.325771,0.613103,0.535725,0.614886,...,0.027848,0.027848,0.493433,0.850772,0.185646,0.357911,0.398330,0.671187,0.205169,0.267424
1815102,田螺,"{""田螺姑娘的故事"": ""0.011"", ""田螺怎么洗最快最干净"": ""0.068"", ""田...",田螺,百科,0.0,0.234428,0.624232,0.268512,0.247083,0.285568,...,0.051282,0.051282,0.322287,0.624232,0.217961,0.319082,0.101104,0.214714,0.008437,0.027830
1815103,经期,"{""经期可以吃什么水果"": ""0.006"", ""经期减肥"": ""0.016"", ""经期可以吃...",经期助手,应用,0.0,0.916361,0.786683,0.809261,0.910182,0.823241,...,0.000000,0.000000,0.849146,0.916361,0.000000,0.875554,0.945661,1.000000,0.407879,0.491249
1815104,花样多,"{""花样多的四字词语"": ""0.082"", ""花样多的成语是什么"": ""0.100"", ""花...",形容花样多的成语,知道,1.0,1.000000,0.407633,0.437123,0.465005,0.853854,...,0.029183,0.023346,0.617250,1.000000,0.358734,0.742338,0.918332,1.000000,0.355184,0.457150


In [53]:
train_data_reset.loc[1815100:1815104]

,prefix,query_prediction,title,tag,label
1815100,习仲勋,"{""习仲勋陵园旅游"": ""0.005"", ""习仲勋故居"": ""0.006"", ""习仲勋陵园""...",习仲勋传,百科,0
1815101,田螺,"{""田螺姑娘的故事"": ""0.011"", ""田螺怎么洗最快最干净"": ""0.068"", ""田...",田螺,百科,0
1815102,经期,"{""经期可以吃什么水果"": ""0.006"", ""经期减肥"": ""0.016"", ""经期可以吃...",经期助手,应用,0
1815103,花样多,"{""花样多的四字词语"": ""0.082"", ""花样多的成语是什么"": ""0.100"", ""花...",形容花样多的成语,知道,1
1815104,本命年,"{""本命年结婚好吗"": ""0.021"", ""本命年是什么意思"": ""0.025"", ""本命年...",本命年可以结婚吗……,健康,0
